<h1>A Survey of Hospital Density in Wisconsin</h1>

<h2>Problem Statement</h2>

When doing initial testing, healthcare manufacturers often choose to do test runs of their equipment at hospitals which are close by. This allows the manufacturer to quickly address any issues that may arise.

However, it is equally important to obtain feedback from users in the field at this stage. To accomplish this, equipment is tested at several local hospitals at a time to maximize feedback.

The goal of this project is to determine which hospitals that a Wisconsin manufacturer should test at in this phase weighing both the density of hospitals in the region and their distance from the manufacturer.

<h2>The Data</h2>

In this notebook, will be using Foursquare to acquire the locations of Wisconsin hospitals. This is used to create a heat-map of hospitals within the state. This information will then be weighed against the distance from the manufacturer to determine which area would be the most productive to test in.

<h2>Code</h2>

<h3>Install Packages if Needed</h3>

In [1]:
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



<h3>Import Libraries</h3>

In [2]:
import requests
import pandas as pd
import numpy as np
import random
import folium

from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

<h3>Data Acquisition</h3>

<b>Manufacturer Information</b>

In [3]:
address = 'Milwaukee, WI'

#get latitude and longitude of the manufacturer address
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print(address, "is located at", latitude, longitude)

Milwaukee, WI is located at 43.0349931 -87.922497


<b>Build the query</b>

In [4]:
# Set the query info
CLIENT_ID = '0LEQXT1Y141HZDYJP5PIUFCCRYTAMQZRABPJOAQ0X5ENU35K' # Foursquare ID
CLIENT_SECRET = 'ADIQGNWNC5JYDXE5I4RF0SCN5VDKYHRRYMOMDK4PQ3DGHUNM' #Foursquare Secret
VERSION = '20180604'
QUERY = 'Hospital'
RADIUS = 10000
LIMIT = 50

In [5]:
# Set the base foursquare api info
foursquare_url = 'https://api.foursquare.com/v2/venues/'
client_info = 'client_id=' + CLIENT_ID + '&client_secret=' + CLIENT_SECRET

In [6]:
# Build the url used to search for hospitals near the manufacturer
url = foursquare_url + 'search?' + client_info
url += '&ll=' + str(latitude) +',' + str(longitude)
url += '&v=' + VERSION
url += '&query=' + QUERY
url += '&radius=' + str(RADIUS)
url += '&limit=' + str(LIMIT)
print("URL generated:\n", url)

URL generated:
 https://api.foursquare.com/v2/venues/search?client_id=0LEQXT1Y141HZDYJP5PIUFCCRYTAMQZRABPJOAQ0X5ENU35K&client_secret=ADIQGNWNC5JYDXE5I4RF0SCN5VDKYHRRYMOMDK4PQ3DGHUNM&ll=43.0349931,-87.922497&v=20180604&query=Hospital&radius=10000&limit=50


<b>Run the Query</b>

In [7]:
results = requests.get(url).json()
if results['meta']['code'] == 200 or results['meta']['code'] == 201:
    df = json_normalize(results['response']['venues'])
    print(len(df), "hospitals found.")
else:
    print("Unable to connect!") 

50 hospitals found.


In [8]:
df = df.rename(columns={'name': 'label', 'location.lat': 'lat', 'location.lng': 'lng'})
hospital_locations = df[['lat', 'lng']]

<h3>Data Analysis</h3>

<b>Assign Hospitals to Clusters</b>

In [9]:
# Fit the coordinates into k groups
k = 5
kmeans = KMeans(n_clusters=k)
kmeans.fit(hospital_locations)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [10]:
# Get group and centroid info
groups = kmeans.labels_
centroids = pd.DataFrame(kmeans.cluster_centers_)

In [11]:
#Convert centroids to Dataframe
colors = {0: 'blue', 1: 'green', 2: 'purple', 3: 'orange', 4:'black'}
i_to_lat_lng = {0: 'lat', 1: 'lng'}

centroids = centroids.rename(columns=i_to_lat_lng)
centroids['color'] = colors.values()

In [12]:
# Add group assignment to the dataframe
grouped_locations = hospital_locations.copy()
grouped_locations['cluster'] = groups

In [13]:
# Convert group number to color
grouped_locations['cluster'] = grouped_locations['cluster'].apply(lambda x: colors[x])

In [14]:
#determine the size of each group
unique, counts = np.unique(grouped_locations['cluster'], return_counts=True)

for i in range(0, len(unique)):
    print("Size of", unique[i], "Group:", counts[i])

Size of black Group: 4
Size of blue Group: 17
Size of green Group: 14
Size of orange Group: 7
Size of purple Group: 8


<b>Determine the distance to each centroid</b>

In [16]:
for i in range(0, len(centroids)):
    d = geodesic((latitude, longitude), (centroids.lat[i], centroids.lng[i]))
    print('The', centroids.color[i], 'group is', d, 'from the manufacturer')

The blue group is 8.157253154745895 km from the manufacturer
The green group is 4.799071137808277 km from the manufacturer
The purple group is 4.496483035549168 km from the manufacturer
The orange group is 2.2296587091479116 km from the manufacturer
The black group is 5.6862190020230505 km from the manufacturer


<h2>Visual Generation<\h2>

<h3>Heat Map of Area Hospitals</h3>

In [17]:
#initialize map
area_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add a red circle marker to represent the manufacturer
folium.features.CircleMarker([latitude, longitude], radius=8, color='red', fill=True, 
                             fill_color='red', fill_opacity=0.6).add_to(area_map)

In [18]:
# add blue circle markers to represent nearby hospitals
for i in range(0, len(hospital_locations)):
    folium.features.CircleMarker([hospital_locations.lat[i], hospital_locations.lng[i]], radius=2,
                                 color='blue', fill=True, fill_color='blue', 
                                 fill_opacity=0.6).add_to(area_map)

In [19]:
#display heat map
area_map

<h3>Clusters</h3>

In [20]:
#initialize map
cluster_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add a red circle marker to represent the manufacturer
folium.features.CircleMarker([latitude, longitude], radius=8, color='red', 
                             fill=True, fill_color='red', fill_opacity=0.6).add_to(cluster_map)

In [21]:
#add centroids by color
#centroids will be darker and have a black outline
for i in range(0, len(centroids)):
    folium.features.CircleMarker([centroids.lat[i], centroids.lng[i]], radius=4,
                                 color='black', fill=True, 
                                 fill_color=centroids.color[i], 
                                 fill_opacity=1).add_to(cluster_map)

In [22]:
#add locations, colored according to their centroid
for i in range(0, len(grouped_locations)):
    folium.features.CircleMarker([grouped_locations.lat[i], grouped_locations.lng[i]], radius=4,
                                 color=grouped_locations.cluster[i], fill=True, 
                                 fill_color= grouped_locations.cluster[i], 
                                 fill_opacity=0.6, stroke=False).add_to(cluster_map)

In [23]:
cluster_map